# Seq2Seq Arabic to English machine translation model

In [ ]:
import pandas as pd
import unicodedata
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.metrics import bleu_score

import nltk
import numpy as np

import random
import math
import time
import unicodedata


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
nltk.download('punkt')
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
    
def tokenize_ar(text):
    """
    Tokenizes Arabic text from a string into a list of strings (tokens) and reverses it
    """
    return [tok for tok in nltk.tokenize.wordpunct_tokenize(unicodeToAscii(text))]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok for tok in nltk.tokenize.wordpunct_tokenize(unicodeToAscii(text))]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('/content/arabic_english.csv')
df= df.drop('Unnamed: 0', axis=1)

In [ ]:
df

In [ ]:
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [ ]:
print(f"Number of training examples: {len(train)}")
print(f"Number of validation examples: {len(validate)}")
print(f"Number of testing examples: {len(test)}")

Number of training examples: 27911
Number of validation examples: 3489
Number of testing examples: 3489


In [ ]:
train.sample(10)

In [ ]:
validate.sample(10)

In [ ]:
test.sample(10)

In [ ]:
def yield_tokens(data_iter, src=True):
    for text in data_iter:
      if src:
        yield tokenize_ar(text)
      else:
        yield tokenize_en(text)

src_vocab = build_vocab_from_iterator(yield_tokens(iter(train['arabic'])),
                                      min_freq=2, 
                                      specials=["<pad>", "<unk>", "<sos>", "<eos>"])
src_vocab.set_default_index(src_vocab["<unk>"])

trg_vocab = build_vocab_from_iterator(yield_tokens(iter(train['english']),src=False), 
                                      min_freq=2, 
                                      specials=["<pad>", "<unk>", "<sos>", "<eos>"])
trg_vocab.set_default_index(trg_vocab["<unk>"])

In [ ]:
torch.save(src_vocab, 'arabic_vocab.pth')
torch.save(trg_vocab, 'english_vocab.pth')

In [ ]:
print(f"Unique tokens in source (ar) vocabulary: {len(src_vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(trg_vocab)}")

Unique tokens in source (ar) vocabulary: 9790
Unique tokens in target (en) vocabulary: 5682


In [ ]:
def preprocess(sequence, vocab, src=True):
        """Converts words to ids."""
        if src:
          tokens = tokenize_ar(sequence.lower())
        else:
          tokens = tokenize_en(sequence.lower())

        sequence = []
        sequence.append(vocab['<sos>'])
        sequence.extend([vocab[token] for token in tokens])
        sequence.append(vocab['<eos>'])
        sequence = torch.Tensor(sequence)
        return sequence

In [ ]:
# from https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
class CustomDataset(Dataset):
    """Custom data.Dataset compatible with data.DataLoader."""
    def __init__(self, src, trg, src_vocab, trg_vocab):
        """Reads source and target sequences from txt files."""
        self.src_seqs = src
        self.trg_seqs = trg
        self.num_total_seqs = len(self.src_seqs)
        self.src_vocab= src_vocab
        self.trg_vocab = trg_vocab

    def __getitem__(self, index):
        """Returns one data pair (source and target)."""
        src_seq = self.src_seqs.iloc[index]
        trg_seq = self.trg_seqs.iloc[index]
        src_seq = self.preprocess(src_seq, self.src_vocab)
        trg_seq = self.preprocess(trg_seq, self.trg_vocab, src=False)
        return src_seq, trg_seq

    def __len__(self):
        return self.num_total_seqs

    def preprocess(self, sequence, vocab, src=True):
        """Converts words to ids."""
        if src:
          tokens = tokenize_ar(sequence.lower())
        else:
          tokens = tokenize_en(sequence.lower())

        sequence = []
        sequence.append(vocab['<sos>'])
        sequence.extend([vocab[token] for token in tokens])
        sequence.append(vocab['<eos>'])
        sequence = torch.Tensor(sequence)
        return sequence


def collate_fn(data):
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    trg_seqs, trg_lengths = merge(trg_seqs) # to cuda


    return src_seqs, src_lengths, trg_seqs, trg_lengths

In [ ]:
def get_loader(src, trg, src_vocab, trg_vocab, batch_size=128):
    dataset = CustomDataset(src, trg, src_vocab, trg_vocab)

    # this will return (src_seqs, src_lengths, trg_seqs, trg_lengths) for each iteration
    data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn)

    return data_loader

In [ ]:
BATCH_SIZE = 16

In [ ]:
train_loader = get_loader(train['arabic'], train['english'], src_vocab, trg_vocab, batch_size=BATCH_SIZE)

validate_loader = get_loader(validate['arabic'], validate['english'], src_vocab, trg_vocab, batch_size=BATCH_SIZE)

test_loader = get_loader(test['arabic'], test['english'], src_vocab, trg_vocab, batch_size=BATCH_SIZE)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]

        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer

        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN

        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))


        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

$$\hat{a}_t = v E_t$$

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a softmax layer.

In [ ]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

  def forward(self, hidden, encoder_outputs):
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]

    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    #repeat decoder hidden state src_len times
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    #hidden = [batch size, src len, dec hid dim]
    #encoder_outputs = [batch size, src len, enc hid dim * 2]

    energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
    #energy = [batch size, src len, dec hid dim]

    attention = self.v(energy).squeeze(2)
    #attention= [batch size, src len]

    return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]

        a = self.attention(hidden, encoder_outputs)
        #a = [batch size, src len]
                
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        weighted = torch.bmm(a, encoder_outputs)
        #weighted = [batch size, 1, enc hid dim * 2]

        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim * 2]

        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]

        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        #prediction = [batch size, output dim]
      
        
        return prediction, hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT).to(device)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn).to(device)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
print(len(src_vocab))
print(len(trg_vocab))

9790
5682


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9790, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5682, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5682, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,581,938 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = trg_vocab['<pad>']).to(device)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = torch.transpose(batch[0], 0,1).to(device)
        trg = torch.transpose(batch[2], 0,1).to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)
        
        #output = [(trg len - 1) * batch size, output dim]
        #trg = [(trg len - 1) * batch size]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = torch.transpose(batch[0], 0,1).to(device)
            trg = torch.transpose(batch[2], 0,1).to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            # trg = trg[1:].view(-1)
            trg = trg[1:].reshape(-1)
        
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, validate_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 2m 12s
	Train Loss: 4.391 | Train PPL:  80.745
	 Val. Loss: 3.956 |  Val. PPL:  52.263
Epoch: 02 | Time: 2m 10s
	Train Loss: 3.107 | Train PPL:  22.356
	 Val. Loss: 3.330 |  Val. PPL:  27.933
Epoch: 03 | Time: 2m 10s
	Train Loss: 2.399 | Train PPL:  11.010
	 Val. Loss: 3.157 |  Val. PPL:  23.505
Epoch: 04 | Time: 2m 10s
	Train Loss: 1.964 | Train PPL:   7.127
	 Val. Loss: 3.119 |  Val. PPL:  22.633
Epoch: 05 | Time: 2m 10s
	Train Loss: 1.658 | Train PPL:   5.246
	 Val. Loss: 3.190 |  Val. PPL:  24.278


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.102 | Test PPL:  22.241 |


Predicting new data

In [ ]:
INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device)

In [ ]:
# model.load_state_dict(torch.load('tut1-model.pt'))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10259, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5682, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5682, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
source = "صباح الخير"

In [ ]:
input = preprocess(source, src_vocab)

In [ ]:
input = input[:,None].to(torch.int64)

In [ ]:
max_length = 10

In [ ]:
target = torch.zeros(max_length,1,).to(torch.int64)

In [ ]:
with torch.no_grad():
     model.eval()
     output = model(input, target, 0)
     output_dim = output.shape[-1]
     output = output[1:].view(-1, output_dim)

In [ ]:
prediction = []
for i in output:
  prediction.append(torch.argmax(i).item())

In [ ]:
tokens = trg_vocab.lookup_tokens(prediction)

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
TreebankWordDetokenizer().detokenize(tokens).replace('<eos>', "").replace('"',"").strip()

'good morning .'